In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/photo-reconstruction/sample_submission.csv
/kaggle/input/photo-reconstruction/Dataset/Training_Data/Dog/Unmasked_Train/Dog-Train (831).jpeg
/kaggle/input/photo-reconstruction/Dataset/Training_Data/Dog/Unmasked_Train/Dog-Train (1381).jpeg
/kaggle/input/photo-reconstruction/Dataset/Training_Data/Dog/Unmasked_Train/Dog-Train (725).jpeg
/kaggle/input/photo-reconstruction/Dataset/Training_Data/Dog/Unmasked_Train/Dog-Train (1366).jpeg
/kaggle/input/photo-reconstruction/Dataset/Training_Data/Dog/Unmasked_Train/Dog-Train (89).jpg
/kaggle/input/photo-reconstruction/Dataset/Training_Data/Dog/Unmasked_Train/Dog-Train (676).jpeg
/kaggle/input/photo-reconstruction/Dataset/Training_Data/Dog/Unmasked_Train/Dog-Train (442).jpeg
/kaggle/input/photo-reconstruction/Dataset/Training_Data/Dog/Unmasked_Train/Dog-Train (582).jpeg
/kaggle/input/photo-reconstruction/Dataset/Training_Data/Dog/Unmasked_Train/Dog-Train (680).jpeg
/kaggle/input/photo-reconstruction/Dataset/Training_Data/Dog/Unmasked_

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
from PIL import Image

In [3]:
import torch

In [4]:
pip install diffusers transformers accelerate scipy safetensors


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 851.8/851.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
def create_mask(filename, dataset):
    y1, x1, y2, x2 = dataset[dataset['filename'] == filename].iloc[0,1:]
    mask = Image.new(mode="RGB", size=(256, 256))
    for i in range(x1, x1+75): 
        for j in range(y1, y1+75):
            mask.putpixel((i,j),(255,255,255)) # for every pixel:
    for i in range(x2, x2+75): 
        for j in range(y2, y2+75):
            mask.putpixel((i,j),(255,255,255)) # for every pixel:
    return(mask)

In [6]:
from diffusers import StableDiffusionInpaintPipeline


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [7]:
pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-inpainting",
    revision="fp16",
    torch_dtype=torch.float16
)
pipe = pipe.to("cuda")


vae/diffusion_pytorch_model.safetensors not found


Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/transformers/models/clip/feature_extraction_clip.py:31: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  FutureWarning,


In [8]:
import cv2 as cv

In [9]:
test_path = '/kaggle/input/photo-reconstruction/Dataset/Testing_Data/'
def submission_data(test_path):
    test_mask_data = pd.read_csv(test_path + 'masked_info.csv').drop(['Unnamed: 0'],axis=1)
    submission = []
    for i in range(len(test_mask_data)):
        if  (i%10)==0:
            print(i)
        filename, y1, x1, y2, x2 = test_mask_data.loc[i,]
        mask = create_mask(filename, test_mask_data)
        mask = mask.resize((512,512))
        masked_image = Image.open(test_path + filename).resize((512,512))
        prompt = "animal"
        image =  pipe(prompt= prompt, image=masked_image, mask_image=mask).images[0]
        image = image.resize((256, 256))
        im = np.array(image)
        b, g, r = cv.split(im)
        result = np.zeros((256, 256))
        normalized_b = cv.normalize(b,result, 0, 1, cv.NORM_MINMAX, dtype=cv.CV_32F)
        normalized_g = cv.normalize(g,result, 0, 1, cv.NORM_MINMAX, dtype=cv.CV_32F)
        normalized_r = cv.normalize(r,result, 0, 1, cv.NORM_MINMAX, dtype=cv.CV_32F)
        for i in range(y1 , y1+75):
            for j in range(x1, x1+75):
                temp1 = filename + '_' + 'box1' + '_' +  str(i) + '_' + str(j) + '_'
                submission.append((temp1 + '0', normalized_b[i,j]))
                submission.append((temp1 + '1', normalized_g[i,j]))
                submission.append((temp1 + '2', normalized_r[i,j]))
        for i in range(y2 , y2+75):
            for j in range(x2, x2+75):
                temp1 = filename + '_' + 'box2' + '_' + str(i) + '_' + str(j) + '_'
                submission.append((temp1 + '0', normalized_b[i,j]))
                submission.append((temp1 + '1', normalized_g[i,j]))
                submission.append((temp1 + '2', normalized_r[i,j]))
        df = pd.DataFrame(submission, columns=['filename_box_pixel','value'])
    return(df)

In [10]:
df = submission_data(test_path)

0


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [11]:
df.head()

,filename_box_pixel,value
0,Tiger (1024).jpeg_box1_58_27_0,0.494118
1,Tiger (1024).jpeg_box1_58_27_1,0.615686
2,Tiger (1024).jpeg_box1_58_27_2,0.294118
3,Tiger (1024).jpeg_box1_58_28_0,0.549020
4,Tiger (1024).jpeg_box1_58_28_1,0.674510


In [12]:
sample = pd.read_csv('/kaggle/input/photo-reconstruction/sample_submission.csv')
sample.shape

(6750000, 2)

In [13]:
test_mask_info = pd.read_csv('/kaggle/input/photo-reconstruction/Dataset/Testing_Data/masked_info.csv').drop(['Unnamed: 0'],axis=1)
test_mask_info.shape

(200, 5)

In [14]:
df.to_csv('/kaggle/input/photo-reconstruction/Dataset/submission.csv')

OSError: [Errno 30] Read-only file system: '/kaggle/input/photo-reconstruction/Dataset/submission.csv'

In [ ]:
df.to_csv('./submission.csv',index=False)